# InCal

In [139]:
import pandas as pd
import plotly.express as px
import numpy as np
from pandas.api.types import CategoricalDtype
from IPython.display import display, HTML
from statsmodels.formula.api import ols
from collections import OrderedDict, Counter
from jupyter_dash import JupyterDash 
from dash import html
from dash import dcc
import itertools
from dash import no_update
from dash import dash_table
import dash
from dash.dependencies import Input, Output, State
from incal_lib import *


# Step 1 - **Upload data**

run cell to upload the data


---



---



In [140]:
dataframes = {}
is_one_file = True
# 'this code block for the jupter colab only'
# try:
#     from google.colab import files
#     import io
#     uploaded = files.upload()
#     if len(uploaded) > 1:
#         for fn in uploaded.keys():
#             print('User uploaded file "{name}" with length {length} bytes'.format(
#                 name=fn, length=len(uploaded[fn])))
#             dataframes[fn] = pd.read_csv(io.BytesIO(uploaded[fn]))
#             is_one_file = False
#     else:
#         name = list(uploaded.keys())[0]
#         dataframes = pd.read_csv(io.BytesIO(uploaded[name]))
# except:
#     print('check the if error - file.csv, there more then one table in the sheet?, is table has missing columns?')

# dataframes = {
#     '3': pd.read_csv("csvs/shani_exp/hebrew_2021-07-28_16_33_hebrew16_shani_w2_acdoors_pt1_m_calr.csv", parse_dates=['Date_Time_1']),
#     '4': pd.read_csv("csvs/shani_exp/hebrew_2021-08-01_13_49_hebrew16_shani_w1_pt2b_m_calr(1).csv", parse_dates=['Date_Time_1']),
#     '5': pd.read_csv("csvs/shani_exp/hebrew_2021-08-04_11_45_hebrew16_shani_acdoors_w2p1_m_calr.csv", parse_dates=['Date_Time_1']),
#     '6': pd.read_csv("csvs/all_weeks/hebrew_2021-08-10_16_15_hebrew16_shani_w2p2.1_m_calr.csv", parse_dates=['Date_Time_1']),
#     '7': pd.read_csv("csvs/all_weeks/hebrew_2021-08-11_16_24_hebrew16_shani_acdoors_w3_m_calr.csv", parse_dates=['Date_Time_1']),
#     '8': pd.read_csv("csvs/all_weeks/hebrew_2021-08-15_16_24_hebrew16_sahni_acdoors_w3p2_m_calr.csv", parse_dates=['Date_Time_1']),
#     '9': pd.read_csv("csvs/all_weeks/hebrew_2021-08-19_16_17_hebrew16_shani_acdoors_w4_m_calr.csv", parse_dates=['Date_Time_1']),
#     '10': pd.read_csv("csvs/all_weeks/hebrew_2021-08-26_16_12_hebrew16_shani_acdoors_w5_m_calr.csv", parse_dates=['Date_Time_1']),
#     '11': pd.read_csv("csvs/all_weeks/hebrew_2021-08-29_08_41_hebrew16_shani_acdoors_w5_dd_m_calr.csv", parse_dates=['Date_Time_1']),
#     '12': pd.read_csv("csvs/all_weeks/hebrew_2021-09-02_07_54_hebrew16_dark dark week2_m_calr.csv", parse_dates=['Date_Time_1'])
# }
dataframes = pd.read_csv('csvs\shani_exp\hebrew_2021-07-28_16_33_hebrew16_shani_w2_acdoors_pt1_m_calr.csv', parse_dates=['Date_Time_1'])
# dataframes = pd.read_csv('example_csvs/example.csv', parse_dates=['Date_Time_1'])
dataframes

,Date_Time_1,envirolightlux_2,envirotemp_2,envirorh_2,envirooccupancy_2,envirosound_2,vo2_1,vo2_2,vo2_3,vo2_4,...,allmeters_7,allmeters_8,allmeters_9,allmeters_10,allmeters_11,allmeters_12,allmeters_13,allmeters_14,allmeters_15,allmeters_16
0,2021-07-28 16:45:00,0,0,0,0,0,1.421349,1.480283,2.206197,1.504177,...,2.371926,4.481019,4.337746,1.404414,3.553978,11.91558,3.425906,4.123155,8.667903,4.39023
1,2021-07-28 17:05:00,0,0,0,0,0,1.587630,1.730032,1.856729,1.248690,...,3.219805,12.191920,7.153311,2.106566,4.424401,16.35851,7.169481,5.093003,18.831850,11.94046
2,2021-07-28 17:25:00,0,0,0,0,0,1.824514,1.503004,1.637191,1.424713,...,3.219805,14.028540,7.280427,2.926421,5.007126,18.59064,8.205385,6.575711,27.916030,15.79637
3,2021-07-28 17:45:00,0,0,0,0,0,1.996406,1.523280,1.403817,1.462108,...,3.253144,14.028540,8.248654,4.176327,6.319085,21.97589,8.214393,8.115648,28.901700,16.33123
4,2021-07-28 18:05:00,0,0,0,0,0,1.707270,1.586915,2.213277,1.421148,...,4.968030,14.311580,11.172880,7.266493,6.572079,21.97760,8.400945,8.585472,28.938640,16.96868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2021-08-01 11:45:00,0,0,0,0,0,1.212794,1.064075,1.124583,1.102364,...,491.530700,633.174600,646.548500,531.221600,699.589700,1047.17600,452.942400,778.114700,762.586900,537.41680
274,2021-08-01 12:05:00,0,0,0,0,0,1.236582,1.054845,1.897775,1.183671,...,491.648800,633.994800,646.972200,532.279500,699.589700,1050.29500,452.944200,782.722100,762.845100,537.72420
275,2021-08-01 12:25:00,0,0,0,0,0,1.308523,1.035974,1.897189,1.386340,...,491.720600,635.166900,648.094000,532.339700,699.589700,1050.35100,452.947800,785.962900,764.462200,537.88200
276,2021-08-01 12:45:00,0,0,0,0,0,1.162044,1.128753,1.476144,1.171823,...,491.814000,635.210900,648.141800,532.339700,700.710300,1050.38200,453.189300,785.962900,764.494900,538.84810


# Step 2 - **INSERT EXPERIMENT DATA**

---



---



In [141]:
dict_groups = OrderedDict(Control = [1, 4, 7, 10, 13], Group_2 = [3, 5, 9, 12, 16], Group_3 = [2, 6, 8, 11, 14, 15])
print(f"{dict_groups}")

OrderedDict([('Control', [1, 4, 7, 10, 13]), ('Group_2', [3, 5, 9, 12, 16]), ('Group_3', [2, 6, 8, 11, 14, 15])])


## names replacement

...

In [142]:
name_for_replacement_in_data_table = {
    'bodymass': 'Weight_(gr)', 
    'vo2': 'Oxygen_Consumption_(ml/hr)', 
    'vco2': 'Carbon_Dioxide_Production_(ml/hr)',
    'kcal_hr': 'Energy_Expenditure_(kcal/hour)',
    'foodupa': 'Cumulative_Food_Intake_(kcal)',
    'rq': 'Respiratory_Exchange_Ratio',
    'pedmeters': 'Pedestrian_Locomotion_(m)',
    'allmeters': 'Total_Distance_includes_fine_movement_(m)',
    'waterupa': 'Cumulative_Water_Intake_(ml)',
    'Wheel': 'Total_Wheel_Counts_(Counts)' # need to check about the wheel parme...
}

calculeted_parmeters = {
    'water': 'Hourly_Water_Intake_(ml)',   
    'food': 'Hourly_Food_Intake_(kcal)',
    'locomotor_activity': 'Locomotor_Activity_(beam_breaks)',
    'energy_balance': 'Energy_Balance_(kcal/hour)',
}

# **Prepring the data**

## Functions for Prepring

-> run cell

## Insert experimental design

In [143]:
unwanted_column = "|".join(['envirolightlux', 'envirooccupancy', 'envirorh', 'envirosound', 'envirotemp'])
cumulative_parm = "|".join(['food', 'water', 'allmeters', 'wheelmeters', 'pedmeters'])
not_for_use_columns = ['vh2o', 'xbreak', 'ybreak', 'index'] # do nothig with it becouse it not importent to delete now this columns
pattern_addition_to_parms = 'actual_'
regx_pattern_for_mean = 'vo2|vco2|vh2o|rq_|bodymass|rq|kcal_hr|break_'
regx_pattern_for_sum = 'water|food'

dict_aggrageted_function_for_column = {
  'Energy_Balance': 'mean',
  'actual_allmeters': 'mean',
  'actual_pedmeters': 'mean',
  'bodymass': 'mean',
  'kcal_hr': 'mean',
  'locomotor_activity': 'mean',
  'rq': 'mean',
  'vco2': 'mean',
  'vo2': 'mean',
  'vh2o': 'mean',
  'xbreak': 'mean',
  'ybreak': 'mean',
  'actual_foodupa': 'sum', 
  'actual_waterupa': 'sum',
}
regx_pattern_no_mean_or_sum = regx_pattern_for_sum + regx_pattern_for_mean

## **Make analysis table**

In [144]:
# Merge files!!
df_or_dfs_in_list = [dataframes] if is_one_file else dataframes.values()
dfs = [incal_get_actuals_from_cumuletive(df, cumulative_parm, pattern_addition_to_parms) for df in df_or_dfs_in_list]
dfs_concated = pd.concat(dfs)

dfs_concated_cleaned = select_columns_by_metebolic_parm(dfs_concated, unwanted_column, True) 

dfs_concated = incal_set_multindex(dfs_concated, ['Date_Time_1']).drop(columns='index')
analysis_format = incal_wide_to_long_df(dfs_concated)

analysis_format[['vco2', 'vh2o', 'vo2']] = analysis_format[['vco2', 'vh2o', 'vo2']].mul(60)
analysis_format[['actual_foodupa']] = analysis_format[['actual_foodupa']].mul(3.56)
analysis_format['Energy_Balance'] = analysis_format['actual_foodupa'].values - analysis_format['kcal_hr'].values
analysis_format['locomotor_activity'] = analysis_format[['xbreak', 'ybreak']].sum(axis=1)

resampled_analysis_format = incal_resample(analysis_format, 'H', dict_aggrageted_function_for_column)

featuers_to_cumsum_by = ['actual_pedmeters', 'actual_allmeters']
analysis_format = incal_cumsum(analysis_format, ['Date_Time_1', 'subjectID'], featuers_to_cumsum_by)
resampled_analysis_format = incal_cumsum(resampled_analysis_format, ['Date_Time_1', 'subjectID'], featuers_to_cumsum_by)

add_feature_for_agg = {
  **dict_aggrageted_function_for_column, 
  'actual_allmeters_cumsum': 'mean',
  'actual_pedmeters_cumsum': 'mean'
}

# analysis format - with original datetime samples
# when waring with datafram less then 15 K rows we can try to use analysis format

# analysis format - datetime agg rolling mean
resampled_analysis_format = incal_assemble_multi_index_format(resampled_analysis_format, 'subjectID', dict_groups, 'Group')
# analysis format - averages for each subject
grouped_analysis_format_df = resampled_analysis_format.groupby(level=['subjectID', 'Group'])
analysis_format_calculeted = grouped_analysis_format_df.agg(add_feature_for_agg).dropna()
# TODO: need to understend why it not average 
analysis_format_calculeted['Energy_Balance'] = analysis_format_calculeted['actual_foodupa'].values - analysis_format_calculeted['kcal_hr'].values

# display(analysis_format.head(2), analysis_format.shape)
display(resampled_analysis_format.head(2), resampled_analysis_format.shape)
display(analysis_format_calculeted.head(2), analysis_format_calculeted.shape)

Energy_Balance  actual_allmeters  \
Date_Time_1         subjectID Group                                       
2021-07-28 16:00:00 1         Control       -0.407835          0.820163   
                    10        Control       -0.088056          1.404414   

                                       actual_allmeters_cumsum  \
Date_Time_1         subjectID Group                              
2021-07-28 16:00:00 1         Control                 0.820163   
                    10        Control                 1.404414   

                                       actual_foodupa  actual_pedmeters  \
Date_Time_1         subjectID Group                                       
2021-07-28 16:00:00 1         Control        0.000000          0.259127   
                    10        Control        0.313275          1.140263   

                                       actual_pedmeters_cumsum  \
Date_Time_1         subjectID Group                              
2021-07-28 16:00:00 1         Control                 0.259127   
                    10        Control                 1.140263   

                                       actual_waterupa  bodymass   kcal_hr  \
Date_Time_1         subjectID Group                                          
2021-07-28 16:00:00 1         Control              0.0  24.46564  0.407835   
                    10        Control              0.0  22.61497  0.401331   

                                       locomotor_activity        rq     vco2  \
Date_Time_1         subjectID Group                                            
2021-07-28 16:00:00 1         Control               201.0  0.758679  64.8669   
                    10        Control               211.0  0.818524  67.9386   

                                           vh2o       vo2  xbreak  ybreak  
Date_Time_1         subjectID Group                                        
2021-07-28 16:00:00 1         Control  8.167320  85.28094    79.0   122.0  
                    10        Control  9.748566  82.76856    64.0   147.0

(1504, 16)

,,Energy_Balance,actual_allmeters,actual_pedmeters,bodymass,kcal_hr,locomotor_activity,rq,vco2,vo2,vh2o,xbreak,ybreak,actual_foodupa,actual_waterupa,actual_allmeters_cumsum,actual_pedmeters_cumsum
subjectID,Group,,,,,,,,,,,,,,,,
1,Control,47.858056,2.995438,2.388813,24.880157,0.460044,355.475177,0.818057,78.149640,94.800892,8.701518,160.127660,195.347518,48.318100,7.847125,140.043900,110.229410
4,Control,57.527601,2.238861,1.551119,25.403154,0.414105,282.255319,0.885638,74.629872,84.131960,8.989577,105.468085,176.787234,57.941706,4.154424,102.655477,69.765315


(16, 16)

# **Dashboard**

In [145]:
def assemble_dash_table(dff):
    return {
        'columns': [{"name": i, "id": i} for i in dff.columns],
        'data': dff.to_dict('records')
    }

def get_start_and_end_time(tuple_start_end, dict_time_stamps):
    start, end = (str(i) for i in tuple_start_end)
    return dict_time_stamps[start], dict_time_stamps[end]

def trim_df_datetime(df, start_time, end_time):
    return df.loc[start_time:end_time]

def assemble_data_to_dom(df, feature_name):
    return df.loc[(), (feature_name, 'bodymass')]

def get_group_or_individual_df(df, category_name, feature_calc):
    time_series_group_or_individual_dfs = {
        'Group': df.groupby([df.index.get_level_values('Date_Time_1'), df.index.get_level_values(category_name)]).agg(feature_calc),
        'subjectID': df
        }
    return time_series_group_or_individual_dfs[category_name]

def graphs_maker(clean_df, df_grouped, group_or_individual_df_averages, averages_df, feature_name, category_name, colors):
    time_series_graph = px.scatter(
        x=df_grouped.index.get_level_values('Date_Time_1'),
        y=df_grouped[feature_name].values,
        color=df_grouped.index.get_level_values(category_name), 
        color_discrete_sequence=colors, 
        template='simple_white'
    ).update_traces(mode='lines+markers')

    histogram = px.histogram(
        x=df_grouped[feature_name].values, 
        color=df_grouped.index.get_level_values(category_name).values, 
        color_discrete_sequence=colors,
        template='simple_white'
        )

    box = px.box(
        x=df_grouped.index.get_level_values(category_name).values,
        y=df_grouped[feature_name].values,
        color=df_grouped.index.get_level_values(category_name).values,
        color_discrete_sequence=colors,
        template='simple_white'
        )
    
    averages = px.bar( # 48
        x=group_or_individual_df_averages.index.get_level_values(category_name),
        y=group_or_individual_df_averages[feature_name].values,
        color=group_or_individual_df_averages.index.get_level_values(category_name),
        color_discrete_sequence=colors, 
        template='simple_white'
    )
    regression = px.scatter(
        x=averages_df['bodymass'].values,
        y=averages_df[feature_name].values,
        color=averages_df.index.get_level_values(1),
        color_discrete_sequence=colors, 
        template='simple_white', 
        trendline='ols'
    )  
    return time_series_graph, histogram, box, averages, regression

In [363]:
resampled_analysis_format.to_csv('csvs/data.csv')

In [146]:
# dashboard code. 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

storage = resampled_analysis_format.copy()
storage_removed_outliears = remove_outliers_mixed_df(storage)


time_stamps = storage.index.get_level_values(0)
shape_analysis_format_indexed = storage.shape
end_point_index_analysis_format_indexed = shape_analysis_format_indexed[0]-1
marks_indexed_time_stamp = {i: time_stamps[i] for i in range(shape_analysis_format_indexed[0])}

list_of_featuers = storage.columns
categories_columns_names = get_categories_cals_names(storage)
obj_categories_columns_names = [{'label': feature, 'value': feature} for feature in categories_columns_names]
obj_features = [{'label': feature, 'value': feature} for feature in list_of_featuers]

subjects = storage.index.get_level_values(1).unique()
groups = storage.index.get_level_values(2).unique()
multi_selection_subjects = [{'label': str(subject), 'value': str(subject)} for subject in subjects]
multi_selection_groups = [{'label': str(group), 'value': str(group)} for group in groups]

levels_as_ids = storage.index
levels_ids, levels_uniques = levels_as_ids.factorize()
columns_names = storage.columns.values.tolist()
def clean_levels_uniques(levels_unique):
  levels_unique = list(levels_unique)
  return " ".join([levels_unique[0].isoformat(), levels_unique[1], levels_unique[2]])
gentrator_id_unique_from_levels = ((id_, unique) for id_, unique in zip(levels_ids, levels_uniques))
multi_select_point = [{
  'label': clean_levels_uniques(levels_unique),
  'value': str(levels_id)} 
  for levels_id, levels_unique in gentrator_id_unique_from_levels]

app.layout = html.Div(children=[
    dcc.Dropdown(
        id='show_as_group_or_individual',
        options=obj_categories_columns_names,
        value=categories_columns_names[0]
    ), 
    dcc.Dropdown(
        id='feature_exp',
        options=obj_features,
        value=list_of_featuers[1]
    ),
    dcc.Checklist(
    id='checklist_remove_outliers',
    options=[
        {'label': 'Remove outliers', 'value': 'True'},
    ],
    value=[],
    labelStyle={'display': 'inline-block'}
    ),
    dcc.RangeSlider(
        marks=marks_indexed_time_stamp,
        value=(0, end_point_index_analysis_format_indexed),
        id="range_slider_time_series", 
        allowCross=False,
        min=0,
        max=end_point_index_analysis_format_indexed,
    ),
    dcc.Dropdown(
        id='remove_group',
        options=multi_selection_groups,
        multi=True
    ), 
    dcc.Dropdown(
        id='remove_subjects',
        options=multi_selection_subjects,
        multi=True
    ), 
    dcc.Dropdown(
        id='state_remove_specific_ids_column',
        options=obj_features,
        multi=True
    ), 
    dcc.Dropdown(
        id='state_remove_specific_ids',
        options=multi_select_point,
        multi=True
    ), 
    html.Button(id='submit_state', n_clicks=0, children='Delete'),
    dcc.Graph(id = 'graph_time_series'), 
    dcc.Graph(id = 'box_graph'), 
    dcc.Graph(id = 'histogram_graph'), 
    dcc.Graph(id = 'graph_averages'), 
    dcc.Graph(id = 'graph_regression'), 
    dash_table.DataTable(id = 'p_values_table')
    # Create element to hide/show, in this case an 'Input Component'
])

@app.callback(
    Output('graph_time_series', 'figure'), 
    Output('box_graph', 'figure'), 
    Output('histogram_graph', 'figure'), 
    Output('graph_averages', 'figure'), 
    Output('graph_regression', 'figure'), 
    Output('p_values_table', 'columns'), 
    Output('p_values_table', 'data'), 
    Input('checklist_remove_outliers', 'value'),
    Input('remove_subjects', 'value'),
    Input('feature_exp', 'value'),
    Input('show_as_group_or_individual', 'value'),
    Input('remove_group', 'value'),
    Input('range_slider_time_series', 'value'), 
    Input('range_slider_time_series', 'marks'), 
    State('state_remove_specific_ids_column', 'value'),
    State('state_remove_specific_ids', 'value'),
    Input('submit_state', 'n_clicks')
)
def pool_data(
    outliers_true, 
    remove_subjects, 
    feature_name, 
    category_name, 
    remove_group, 
    tuple_start_end, 
    dict_time_stamps, 
    state_remove_specific_ids_column,
    state_remove_specific_ids, 
    submit_state_n_clicks
    ):

    data = storage.copy() if not outliers_true else storage_removed_outliears.copy()
    
    if submit_state_n_clicks: # idealy the if else need to help the user to work with the buttons 
        levels = [levels_uniques[int(number_string)] for number_string in state_remove_specific_ids]
        for level, column in zip(levels, state_remove_specific_ids_column):
            data.at[level, column] = np.nan

    start_time, end_time = get_start_and_end_time(tuple_start_end, dict_time_stamps)
    clean_df = trim_df_datetime(
        data,
        start_time, 
        end_time
        )
    


    if remove_group:
        clean_df = incal_remove_group(clean_df, 2, remove_group)

    if remove_subjects:
        clean_df = incal_remove_subjects(clean_df, 1, remove_subjects)
    
    # data to dom
    data_for_view = assemble_data_to_dom(clean_df, feature_name)
    # time series
    feature_calc = add_feature_for_agg[feature_name]
    df_grouped = get_group_or_individual_df(data_for_view, category_name, feature_calc)
    # averages
    group_or_individual_df_averages = data_for_view.groupby([data_for_view.index.get_level_values(category_name)]).agg(feature_calc)
    # regression
    averages_df = clean_df.groupby([clean_df.index.get_level_values('subjectID'), clean_df.index.get_level_values('Group')]).agg(feature_calc).dropna()
    # make graphs
    colors = px.colors.qualitative.Vivid
    time_series_graph, histogram, box, averages, regression = graphs_maker(
        clean_df,
        df_grouped,
        group_or_individual_df_averages, 
        averages_df, 
        feature_name, 
        category_name, 
        colors
        )
    # need to refactore that remove the side affect of in this function 
    grouped_analysis_format_df = clean_df.groupby(categories_columns_names)
    analysis_format_calculeted = grouped_analysis_format_df.agg(add_feature_for_agg).dropna()
    analysis_format_calculeted['Energy_Balance'] = analysis_format_calculeted['actual_foodupa'].values - analysis_format_calculeted['kcal_hr'].values
    analysis_format_calculeted_index_reseted = analysis_format_calculeted.reset_index()
    p_values_table = create_anovas_table(analysis_format_calculeted_index_reseted)
    p_values_table = p_values_table.reset_index().rename(columns={'index':'Features'})
    columns = [{'id': p, 'name': p} for p in p_values_table.columns.to_list()]
    return time_series_graph, box, histogram, averages, regression, columns, p_values_table.to_dict('records')

In [147]:
def run_server(self,
               port=8004,
               debug=True,
               threaded=True,
               **flask_run_options):
    self.server.run(port=port, debug=debug, **flask_run_options)

if __name__ == '__main__':
    app.run_server(debug=True, port=8024, mode='external')

Dash app running on http://127.0.0.1:8024/


c:\Users\misha\Desktop\Projects\InCal\venv\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [148]:
# http://127.0.0.1:8024/

In [149]:
l1 = []
l2 = []


l1.append('11')
l1.append('11')
l2.append(('_11_', 'm'))
l2.append(('_11_', 'm'))




In [150]:
l2, l1


([('_11_', 'm'), ('_11_', 'm')], ['11', '11'])

In [262]:
dicty = {(1+v): np.random.Generator(np.random.PCG64()).standard_normal() for v in range(10)}

In [263]:
my_data = pd.DataFrame(dicty, index=pd.MultiIndex.from_tuples([(i, 1+i*2, 2+i*3) for i in range(10)]))

In [264]:
my_data

,,,1,2,3,4,5,6,7,8,9,10
0,1,2,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
1,3,5,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
2,5,8,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
3,7,11,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
4,9,14,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
5,11,17,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
6,13,20,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
7,15,23,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
8,17,26,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836
9,19,29,2.40632,0.054768,0.512382,1.52019,1.137796,-2.058813,-1.025542,-1.248795,-0.607619,-0.320836


In [355]:
storage_key_and_vlaue = {}
ids_list_storage = []

list_after, feature = ['0', '2', '6', '9'], 1
list_after, feature = ['2', '6', '9', '9'], 3
# ############### function for deleting specifc value
def eval_who_to_add_and_dalete(list_before, list_after):
  storage_addings, storage_daleteing = set(list_after) - set(list_before), set(list_before) - set(list_after)
  return storage_addings, storage_daleteing

def change_value_in_df_at_col_row(df, value=np.nan):
  my_data.at[ids] = value

def addings_to_storage(df, storage, adds, feature):
  for item in items:
    row = df.index[int(item)] # add: int type
    value = df.at[row, (feature)]
    index, row, feature = (i, row, feature)
    storage[(index, row, feature)] = value

def keys_to_remove(df, storage, delete):
  return [i for i in storage__.keys() if i[0] in delete]

# storage list update
def remove_item_from_storage_ids_list(ids_list_storage, to_filter_out):
  return list(filter(lambda item: item not in to_filter_out, ids_list_storage))
def extend_storage(ids_list, adds):
  return [*ids_list, *adds]
def update_storage_ids_list(ids_list_storage, new_list):
  add, del_ = eval_who_to_add_and_dalete(ids_list_storage, list_after)
  ids_list_storage = remove_item_from_storage_ids_list(ids_list_storage, del_)
  return extend_storage(ids_list_storage, add)

def remove_from_storage(df, storage, remove_key):
  del storage__[remove]
# ############### end of - functions for deleting specifc value 



In [359]:
storage_key_and_vlaue

{(6, (0, 1, 2), 4): 1.520190104061012,
 (6, (4, 9, 14), 4): 1.520190104061012,
 (6, (5, 11, 17), 4): 1.520190104061012}

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [ ]:
l2, feature2 = ['0', '2', '3', '8'], 3
add, delete = add_dalete(list_after, l2)